In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import copy

In [2]:
def harris(filepath, blockSize=2, ksize=3, k=0.004):
    """
    # blockSize (np. 2) - Jest to wielkość sąsiedztwa brana pod uwagę przy wykrywaniu narożników
    blockSize - It is the size of neighbourhood considered for corner detection
    # ksize (np. 3) - Parametr wielkości maksi używanej pochodnej Sobela.
    ksize - Aperture parameter of the Sobel derivative used.
    # k (np. 0.004) - Parametr wolny w równaniu detektora Harris (0.004 – 0.006)
    k - Harris detector free parameter in the equation.
    
    https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html
    """
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_grey = np.float32(img_grey)
    harris = cv2.cornerHarris(img_grey, blockSize, ksize, k)
    img[harris > 0.1 * harris.max()] = [0, 0, 255]
    # co tu się dzieje? co oznacza harris max? zo zwraca harris? czemu to nie jest wyjaśniane.... jprdlsvd kjbsdbkgfjjkdfa
    cv2.imshow('Punkty wykryte detektorem Harrisa', img)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()
    return img  #, harris? jak dostac wsp... 

In [3]:
def harris_extended(filepath, blockSize=2, ksize=3, k=0.004, display_result=False):
    """
    # blockSize (np. 2) - Jest to wielkość sąsiedztwa brana pod uwagę przy wykrywaniu narożników
    blockSize - It is the size of neighbourhood considered for corner detection
    # ksize (np. 3) - Parametr wielkości maksi używanej pochodnej Sobela.
    ksize - Aperture parameter of the Sobel derivative used.
    # k (np. 0.004) - Parametr wolny w równaniu detektora Harris (0.004 – 0.006)
    k - Harris detector free parameter in the equation.
    
    https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html
    """
    # Konwersja obrazu do postaci zdjęcia w odcieniach szarości
    zdjecie = cv2.imread(filepath)
    szareZdjecie = cv2.cvtColor(zdjecie, cv2.COLOR_BGR2GRAY) 

    # find Harris corners
    szareZdjecie = np.float32(szareZdjecie)
    harris = cv2.cornerHarris(szareZdjecie, blockSize, ksize, k)
    harris = cv2.dilate(harris, None)
    ret, dst = cv2.threshold(harris, 0.01 * harris.max(), 255, 0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(szareZdjecie, np.float32(centroids), (5, 5), (-1, -1), criteria)

    # Now draw them
    res = np.hstack((centroids, corners))
    res = np.int0(res)
    zdjecie[res[:, 1], res[:, 0]] = [0, 0, 255] # image = cv2.circle(image, (int(v[0]),int(v[1])), radius=2, color=(0, 0, 255), thickness=5)
    zdjecie[res[:, 3], res[:, 2]] = [0, 255, 0]

    if display_result:
        cv2.imshow('Punkty wykryte detektorem Harrisa', zdjecie)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()

    return corners, zdjecie

In [4]:
def harris_extended_in_array(input_array, blockSize=2, ksize=3, k=0.004, threshold=0.01):
    """
    input_array - np.ndarray(dtype="float32") reprezentująca zdjęcie w odcieniach szarości
    # blockSize (np. 2) - Jest to wielkość sąsiedztwa brana pod uwagę przy wykrywaniu narożników
    blockSize - It is the size of neighbourhood considered for corner detection
    # ksize (np. 3) - Parametr wielkości maksi używanej pochodnej Sobela.
    ksize - Aperture parameter of the Sobel derivative used.
    # k (np. 0.004) - Parametr wolny w równaniu detektora Harris (0.004 – 0.006)
    k - Harris detector free parameter in the equation.
    # threshold - odsyłam do dokumentacji
    
    https://docs.opencv.org/3.4/dc/d0d/tutorial_py_features_harris.html
    """
    # find Harris corners
    harris = cv2.cornerHarris(input_array, blockSize, ksize, k)
    harris = cv2.dilate(harris, None)
    ret, dst = cv2.threshold(harris, threshold * harris.max(), 255, 0)
    dst = np.uint8(dst)

    # find centroids
    ret, labels, stats, centroids = cv2.connectedComponentsWithStats(dst)

    # define the criteria to stop and refine the corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 0.001)
    corners = cv2.cornerSubPix(input_array, np.float32(centroids), (5, 5), (-1, -1), criteria)

    return corners, centroids

In [5]:
def fast_stock(filepath):
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Inicjalizacja detektora FAST z wartościami domyślnymi funkcji
    fast = cv2.FastFeatureDetector.create()
    # Wykrycie punktów charakterystycznych (keypoints)
    kp = fast.detect(img_grey, None)
    # Narysowanie punktów charakterystycznych (keypoints)
    img2 = cv2.drawKeypoints(img_grey, kp, None, color=(255, 0, 0))
    # Wyświetlenie parametrów funkcji
    print("Threshold: ", fast.getThreshold())
    print("nonmaxSuppression: ", fast.getNonmaxSuppression())
    print("neighborhood: ", fast.getType())
    print("Total Keypoints without nonmaxSuppression: ", len(kp))
    cv2.imshow('Punkty wykryte algorytmem Fast', img2)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()
    return img2 #, 

In [6]:
def fast(filepath, threshold=10, nonmaxSuppresion=True, type=cv2.FAST_FEATURE_DETECTOR_TYPE_9_16):
    """
    FAST: Features from  Accelerated  Segment  Test
    cv2.FAST_FEATURE_DETECTOR_TYPE_9_16, cv2.FAST_FEATURE_DETECTOR_TYPE_5_8, cv2.FAST_FEATURE_DETECTOR_TYPE_7_12
    """
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fast = cv2.FastFeatureDetector.create(threshold, nonmaxSuppresion, type)
    keypoints = fast.detect(img_grey, None)
    img2 = cv2.drawKeypoints(img_grey, keypoints, None, color=(255, 0, 0))
    # cv2.imshow('Punkty wykryte algorytmem Fast', img2)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()
    return img2, keypoints # . . .

In [7]:
def sift(img_filepath, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, display_result=False):
    """
    Parameters
        img_filepath     filepath to RGB image
        nfeatures	The number of best features to retain. The features are ranked by their scores (measured in SIFT algorithm as the local contrast)
        nOctaveLayers	The number of layers in each octave. 3 is the value used in D. Lowe paper. The number of octaves is computed automatically from the image resolution.
        contrastThreshold	The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. The larger the threshold, the less features are produced by the detector.
        edgeThreshold	The threshold used to filter out edge-like features. Note that the its meaning is different from the contrastThreshold, i.e. the larger the edgeThreshold, the less features are filtered out (more features are retained).
        sigma	The sigma of the Gaussian applied to the input image at the octave #0. If your image is captured with a weak camera with soft lenses, you might want to reduce the number.
    Note
        The contrast threshold will be divided by nOctaveLayers when the filtering is applied. When nOctaveLayers is set to default (3) and if you want to use the value used in D. Lowe paper, 0.03, set this argument to 0.09.
    
    # https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html
    # https://docs.opencv.org/4.x/d7/d60/classcv_1_1SIFT.html
    """
    img = cv2.imread(img_filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)
    keypoints = sift.detect(img_grey, None)
    zdjecie = cv2.drawKeypoints(img, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    if display_result:
        cv2.imshow(f"Detektor SIFT, {nfeatures=} {nOctaveLayers=} {contrastThreshold=} {edgeThreshold=} {sigma=}", zdjecie)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()
    return zdjecie, keypoints

In [8]:
def sift_in_array(input_array, nfeatures=0, nOctaveLayers=3, contrastThreshold=0.04, edgeThreshold=10, sigma=1.6, display_result=False):
    """
    Parameters
        input_array     np.ndarray represening an image in greyscale
        nfeatures	The number of best features to retain. The features are ranked by their scores (measured in SIFT algorithm as the local contrast)
        nOctaveLayers	The number of layers in each octave. 3 is the value used in D. Lowe paper. The number of octaves is computed automatically from the image resolution.
        contrastThreshold	The contrast threshold used to filter out weak features in semi-uniform (low-contrast) regions. The larger the threshold, the less features are produced by the detector.
        edgeThreshold	The threshold used to filter out edge-like features. Note that the its meaning is different from the contrastThreshold, i.e. the larger the edgeThreshold, the less features are filtered out (more features are retained).
        sigma	The sigma of the Gaussian applied to the input image at the octave #0. If your image is captured with a weak camera with soft lenses, you might want to reduce the number.
    Note
        The contrast threshold will be divided by nOctaveLayers when the filtering is applied. When nOctaveLayers is set to default (3) and if you want to use the value used in D. Lowe paper, 0.03, set this argument to 0.09.
    
    # https://docs.opencv.org/3.4/da/df5/tutorial_py_sift_intro.html
    # https://docs.opencv.org/4.x/d7/d60/classcv_1_1SIFT.html
    """
    sift = cv2.SIFT_create(nfeatures, nOctaveLayers, contrastThreshold, edgeThreshold, sigma)
    keypoints = sift.detect(input_array, None)
    just_coords = [p.pt for p in kp]
    zdjecie = cv2.drawKeypoints(input_array, keypoints, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    if display_result:
        cv2.imshow(f"Detektor SIFT, {nfeatures=} {nOctaveLayers=} {contrastThreshold=} {edgeThreshold=} {sigma=}", zdjecie)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()
    return keypoints, just_coords, zdjecie

In [9]:
def mouse_callback_function(event, x, y, flags, param):
    global pixel_coords_4_points  # lista globalna
    if event == cv2.EVENT_LBUTTONDOWN:  # cv2.EVENT_LBUTTONDOWN EVENT_LBUTTONDBLCLK
        pixel_coords_4_points.append((x, y))
        print(pixel_coords_4_points)
    if len(pixel_coords_4_points)==4:
        cv2.destroyAllWindows()

In [10]:
def warpPoints(list_of_points, transformation_matrix):
    """
    warpPerspective for list of 2D points
    Inspired by https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87

    Args:
        list_of_points (list): list of 2-element tuples eg.: [(0,0), (-2,-4), (7,7)]
        transformation_matrix (np.ndarray): matrix used for transoformation according to equation in https://docs.opencv.org/3.4/da/d54/group__imgproc__transform.html#gaf73673a7e8e18ec6963e3774e6a94b87

    Returns:
        list_of_transformed_points
    """
    list_of_transformed_points = []
    for p in list_of_points:
        tp = ((transformation_matrix[0,0]*p[0] + transformation_matrix[0,1]*p[1] + transformation_matrix[0,2])/(transformation_matrix[2,0]*p[0] + transformation_matrix[2,1]*p[1] + transformation_matrix[2,2]),
              (transformation_matrix[1,0]*p[0] + transformation_matrix[1,1]*p[1] + transformation_matrix[1,2])/(transformation_matrix[2,0]*p[0] + transformation_matrix[2,1]*p[1] + transformation_matrix[2,2]))
        list_of_transformed_points.append(tp)
    return list_of_transformed_points

In [225]:
def get_corresponding_points(img_path, scale_percent=15, gridsize=20, squares_w=7, squares_h=7, square_side=4.8, snap_threshold=1.2, haris_threshold=0.01, show_detected_pts=True, show_filtered_detected_pts=True, show_grid=True):
    """Nie chce mi się pisać docstringa.
    Przed wywołaniem musi być zainicjalizowana globalna lista pixel_coords_4_points = [] 

    Args:
        img_path (str): Image filepath.
        scale_percent (float): Img resize percent.
        gridsize (int, optional): Defaults to 20. Ile oczek siatki w górę, w dół, w prawo, w lewo od punktu centralnego siatki.
        squares_w (int, optional): Defaults to 7. Szerokość w kwadratach prostokąta klikanego (klikanych czterech narożników).
        squares_h (int, optional): Defaults to 7. Wysokość w kwadratach prostokąta klikanego (klikanych czterech narożników).
        square_side (float, optional): Defaults to 4.8. Długość boku jednego kwadratu (pola szachownicy).
        snap_threshold (float, optional): Defaults to 1.2.
        haris_threshold (float, optional): Defaults to 0.01.

    Returns:
        (List, List): (our_xyz_points_transformed, closest_detected_points)
    """
    img = cv2.imread(img_path)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    img_resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)
    img_grey_resized = cv2.resize(img_grey, (width, height), interpolation=cv2.INTER_CUBIC)


    # pixel_coords_4_points = [] 
    def mouse_callback_function(event, x, y, flags, param):
        global pixel_coords_4_points  # lista globalna
        if event == cv2.EVENT_LBUTTONDOWN:
            pixel_coords_4_points.append((x, y))
            print(pixel_coords_4_points)
        if len(pixel_coords_4_points)==4:
            cv2.destroyAllWindows()

    # Klikamy cztery narożniki tworzące prostokąt squares_w x squares_h (default 7x7) w ustalonej kolejkności: LU RU RD LD
    # pixel_coords_4_points = [] 
    cv2.namedWindow("Click four corners")
    cv2.setMouseCallback("Click four corners", mouse_callback_function)
    cv2.imshow("Click four corners", img_resized)
    if cv2.waitKey(0):
        cv2.destroyAllWindows()

    # Kliknięte współrzędne pikselowe zapisywane są w liście pixel_coords_4_points
    global pixel_coords_4_points
    pixel_coords_4_points = np.array(pixel_coords_4_points, dtype=np.dtype('float32'))

    # Tworzymy nasz układ współrzędnych (wybieramy cztery punkty odpowiadające klikniętym). (0,0) w lewym górnym kliniętym narożniku.
    our_coords_4_points = [(0, 0), (square_side*squares_w, 0), (square_side*squares_w, square_side*squares_h), (0, square_side*squares_h)]
    our_coords_4_points = np.array(our_coords_4_points, dtype=np.dtype('float32'))

    # Macierz do transformacji z układu wsp. pikselowych do naszego układu wsp.
    macierz_transformacji = cv2.getPerspectiveTransform(pixel_coords_4_points, our_coords_4_points)

    # Macierz do odwrotnej transformacji (z naszego układu wsp. do układu wsp. pikselowych)
    macierz_odwrotna = np.linalg.inv(macierz_transformacji)

    # Robimy siatkę/listę punktów w naszym układzie współrzędnych
    x = np.arange(-gridsize*4.8, gridsize*4.8, square_side)
    y = np.arange(-gridsize*4.8, gridsize*4.8, square_side)
    xx, yy = np.meshgrid(x, y)
    our_xyz_points = np.hstack([np.expand_dims(xx.ravel(),axis=0).T, np.expand_dims(yy.ravel(),axis=0).T, np.expand_dims(np.repeat(1, yy.size),axis=0).T]) 
    our_xy_points = our_xyz_points[:, :2]

    # Wykrycie punktów narożnych detektorem harrisa
    detected_points, centroids = harris_extended_in_array(img_grey_resized, threshold=haris_threshold)

    # Wyświetlenie wykrytych narożników
    if show_detected_pts:
        image = copy.deepcopy(img_resized)
        for v in detected_points:
            image = cv2.circle(image, (int(v[0]),int(v[1])), radius=0, color=(0, 0, 255), thickness=5)
        cv2.imshow('Detected points (harris detector)', image)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()

    # Transformacja wykrytych narożników do naszego układu współrzędnych
    detected_points_in_our_coords = warpPoints(detected_points, macierz_transformacji)

    # Dla każdego punktu z naszej siatki wykrycie najbliższego punktu wykrytego detektorem 
    closest_detected_points_in_our_coords = []
    our_xy_points_2 = []
    for p in our_xy_points:
        distances = np.linalg.norm(detected_points_in_our_coords - p, axis=1)
        min_index = np.argmin(distances)
        if distances[min_index] <= snap_threshold:
            closest_detected_points_in_our_coords.append(detected_points_in_our_coords[min_index])
            our_xy_points_2.append(p)

    # Transformacja punktów z detektora (najbliższych siatce) do ich orginalnego pikselowego układu współrzędnych zdjęcia
    closest_detected_points = warpPoints(closest_detected_points_in_our_coords, macierz_odwrotna)
    
    # Wyświetlenie odfiltrowanych punktów z detektora
    if show_filtered_detected_pts:
        image = copy.deepcopy(img_resized)
        for v in closest_detected_points:
            image = cv2.circle(image, (int(v[0]),int(v[1])), radius=0, color=(0, 0, 0), thickness=5)
        cv2.imshow('closest detected points (harris detector)', image)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()

    # Transformacja siatki z naszego układu do układu pikselowego zdjęcia
    our_xy_points_transformed = warpPoints(our_xy_points_2, macierz_odwrotna)

    # Wyświetlenie przetransformowanych punktów grid
    if show_grid:
        image = copy.deepcopy(img_resized)
        for v in our_xy_points_transformed:
            image = cv2.circle(image, (int(v[0]),int(v[1])), radius=0, color=(150, 0, 50), thickness=5)
        cv2.imshow('our xy points transformed', image)
        if cv2.waitKey(0) & 0xff == 27:
            cv2.destroyAllWindows()

    img_grey_resized_size = img_grey_resized.shape[::-1]
    our_xyz_points_transformed = np.hstack((np.array(our_xy_points_transformed), np.zeros(shape=(len(our_xy_points_transformed),1)))).astype('float32')  #.tolist()
    # return our_xyz_points_transformed, closest_detected_points, img_grey_resized_size

    # TODO nwm co sie dzieje... CO TRZEBA ZWRACAĆ?
    our_points = []
    for pair in our_xy_points_2:
        our_points.append((pair[0], pair[1], 0))
    our_points = np.array(our_points, dtype=np.dtype("float32"))
    detected_points = np.expand_dims(np.array(closest_detected_points, dtype=np.dtype("float32")), axis=1)
    return our_points, detected_points, img_grey_resized_size

# Dane wejściowe (folder ze zdjęciami) ```*można pominąć tę sekcję, a na końcu usunąć*```
i przygotowanie list z macierzami reprezentującymi zdjęcia (rgb/grey, orginalny_rozmiar/resize)

In [14]:
# Wczytanie z folderu obrazków w rgb i obrazków skali szarości (jako macierze) do list: images i images_grey
# board_images_folder = 'D:\\STUDIA_6_SEM\\PCPO\\zdj_do_kalibracji_1'
board_images_folder = r"C:\SEM6\PCPO\p9\images"
files = [f for f in listdir(board_images_folder) if isfile(join(board_images_folder, f))]
images = []
images_grey = []
for filename in files:
    filepath = rf"{board_images_folder}\\{filename}"
    img = cv2.imread(filepath)
    img_grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    images.append(img)
    images_grey.append(img_grey)

In [15]:
# Resize obrazów i wczytanie ich do list: images_resized i images_grey_resized
images_resized = []
images_grey_resized = []
scale_percent = 15 # percent of original size
for img_grey, img in zip(images_grey, images):
    width = int(img.shape[1] * scale_percent / 100)
    height = int(img.shape[0] * scale_percent / 100)
    img_resized = cv2.resize(img, (width, height), interpolation=cv2.INTER_CUBIC)
    img_grey_resized = cv2.resize(img_grey, (width, height), interpolation=cv2.INTER_CUBIC)
    images_grey_resized.append(img_grey_resized)
    images_resized.append(img_resized)

In [16]:
# Wyświetlenie obrazków z wybranej listy
lista = images_resized
for i, img in enumerate(lista):
    cv2.imshow(f'image {i+1} out of {len(lista)}', img)
    if cv2.waitKey(0) & 0xff == 27:
        cv2.destroyAllWindows()

# TEST

In [229]:
import glob
images = glob.glob(r"C:\SEM6\PCPO\p9\images_2\\*.png")
results_folder = r"C:\SEM6\PCPO\p9\images_2\undistorted\\"
# images = glob.glob(r"C:\SEM6\PCPO\p9\images\\*.jpg")
# results_folder = r"C:\SEM6\PCPO\p9\images\undistorted\\"
images

['C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (1).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (10).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (11).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (2).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (3).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (4).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (5).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (6).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (7).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (8).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image (9).png',
 'C:\\SEM6\\PCPO\\p9\\images_2\\MicrosoftTeams-image.png']

In [227]:
objpoints = []
imgpoints = []
for img_path in images:
    pixel_coords_4_points = [] 
    our, detected, size = get_corresponding_points(img_path, scale_percent=70, gridsize=100, squares_w=15, squares_h=23)
    objpoints.append(our)
    imgpoints.append(detected)

[(411, 296)]
[(411, 296), (929, 297)]
[(411, 296), (929, 297), (892, 963)]
[(411, 296), (929, 297), (892, 963), (472, 941)]
[(515, 389)]
[(515, 389), (752, 535)]
[(515, 389), (752, 535), (764, 1098)]
[(515, 389), (752, 535), (764, 1098), (496, 1145)]
[(267, 384)]
[(267, 384), (671, 414)]
[(267, 384), (671, 414), (597, 1293)]
[(267, 384), (671, 414), (597, 1293), (284, 1049)]
[(488, 199)]
[(488, 199), (860, 153)]
[(488, 199), (860, 153), (854, 889)]
[(488, 199), (860, 153), (854, 889), (499, 797)]
[(553, 140)]
[(553, 140), (924, 195)]
[(553, 140), (924, 195), (919, 806)]
[(553, 140), (924, 195), (919, 806), (575, 922)]
[(268, 237)]
[(268, 237), (704, 203)]
[(268, 237), (704, 203), (779, 878)]
[(268, 237), (704, 203), (779, 878), (278, 915)]
[(302, 416)]
[(302, 416), (666, 472)]
[(302, 416), (666, 472), (661, 1060)]
[(302, 416), (666, 472), (661, 1060), (311, 1193)]
[(595, 205)]
[(595, 205), (1004, 201)]
[(595, 205), (1004, 201), (1060, 871)]
[(595, 205), (1004, 201), (1060, 871), (514, 

In [205]:
for array in objpoints:
    print(array.shape)
for array in imgpoints:
    print(array.shape)

(166, 3)
(145, 3)
(155, 3)
(142, 3)
(100, 3)
(95, 3)
(75, 3)
(119, 3)
(144, 3)
(123, 3)
(141, 3)
(84, 3)
(166, 1, 2)
(145, 1, 2)
(155, 1, 2)
(142, 1, 2)
(100, 1, 2)
(95, 1, 2)
(75, 1, 2)
(119, 1, 2)
(144, 1, 2)
(123, 1, 2)
(141, 1, 2)
(84, 1, 2)


In [228]:
ret, cameraMatrix, dist, rvecs, tvecs = cv2.calibrateCamera(objectPoints=objpoints, imagePoints=imgpoints, imageSize=size, cameraMatrix=None, distCoeffs=None)
ret, cameraMatrix, dist, rvecs, tvecs

(3.5588502364509678,
 array([[1.11688774e+03, 0.00000000e+00, 6.19654247e+02],
        [0.00000000e+00, 1.11131557e+03, 6.36953524e+02],
        [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]),
 array([[-0.22454995,  0.4165418 ,  0.00621792, -0.00659081, -0.29873302]]),
 (array([[ 0.38897692],
         [ 0.15845153],
         [-0.0050568 ]]),
  array([[-0.28136919],
         [-0.91968708],
         [ 0.14474734]]),
  array([[0.2141417 ],
         [0.69949942],
         [0.14601798]]),
  array([[0.20786007],
         [0.63458022],
         [0.05901014]]),
  array([[ 0.18452532],
         [-0.55585404],
         [-0.08277803]]),
  array([[-0.18833798],
         [-0.01282422],
         [-0.07958083]]),
  array([[ 0.00964902],
         [-0.75938252],
         [-0.00250752]]),
  array([[-0.35537701],
         [ 0.14219347],
         [ 0.05779587]]),
  array([[ 0.12318729],
         [-0.04598486],
         [-0.00707828]]),
  array([[ 0.02126299],
         [-0.07645614],
         [-0.02059

In [232]:
for img_path in images:
    img = cv2.imread(img_path)

    # scale_percent = 15
    # w = int(img.shape[1] * scale_percent / 100)
    # h = int(img.shape[0] * scale_percent / 100)
    h, w = img.shape[:2]

    newCameraMatrix, roi = cv2.getOptimalNewCameraMatrix(cameraMatrix, dist, (w, h), 1, (w, h))

    # undistort
    dst = cv2.undistort(img, cameraMatrix, dist, None, newCameraMatrix)

    # # crop the image
    # x, y, w, h = roi
    # dst = dst[y : y + h, x : x + w]
    
    name = img_path[img_path.rfind('\\')+1 : ]
    cv2.imwrite(results_folder + name, dst)


Nie wiem, czy dobrze działa ta kalibracja kamery (wyniki po undistort)

Jakie listy zwracać w get_corresponding_points? Czym są argumenty objectPoints i imagePoints w cv2.calibrateCamera()?